In [20]:
import pandas as pd
import mysql.connector

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras. Una de las tablas contendrá la información que obtuvimos de los ejercicios de pair programming de Limpieza, es decir, el data set de ataques de tiburones limpito. La segunda tabla tendrá la información obtenida en el ejercicio de pair de ETL 1.

1- Cread la BBDD con el nombre de tiburones.

In [21]:
df_ataques = pd.read_csv('../Ficheros/attack_limpieza5.csv', index_col= 0)
df_clima = pd.read_csv('../Ficheros/ETL1.csv', index_col= 0)

In [22]:
df_ataques.head(3)

,case_number,year,country,activity,age,species_,date,mes,fatal_,sex2,nuevas_edades
0,1800.00.00,1977,seychelles,a corsairs boat was overturned,NaN,NaN,1800,NaN,y,F,27.301694
1,1797.05.28.R,1977,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,y,M,27.301694
2,1792.09.12,1977,england,Fishing,NaN,NaN,12-Sep-1792,Sep,y,M,27.301694


In [23]:
df_ataques.dtypes

case_number       object
year               int64
country           object
activity          object
age               object
species_          object
date              object
mes               object
fatal_            object
sex2              object
nuevas_edades    float64
dtype: object

In [24]:
df_ataques.isnull().sum()

case_number         1
year                0
country            50
activity            0
age              2820
species_         2831
date                0
mes               570
fatal_            607
sex2                0
nuevas_edades       0
dtype: int64

In [25]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 175, 'speed':...",26,2,6,1011,none,0,0,130,2,39.78373,-100.445882,USA


In [26]:
df_clima.dtypes

timepoint              int64
cloudcover             int64
highcloud              int64
midcloud               int64
lowcloud               int64
rh_profile            object
wind_profile          object
temp2m                 int64
lifted_index           int64
rh2m                   int64
msl_pressure           int64
prec_type             object
prec_amount            int64
snow_depth             int64
wind10m.direction      int64
wind10m.speed          int64
latitud              float64
longitud             float64
paises                object
dtype: object

In [27]:
df_clima.isnull().sum()

timepoint            0
cloudcover           0
highcloud            0
midcloud             0
lowcloud             0
rh_profile           0
wind_profile         0
temp2m               0
lifted_index         0
rh2m                 0
msl_pressure         0
prec_type            0
prec_amount          0
snow_depth           0
wind10m.direction    0
wind10m.speed        0
latitud              0
longitud             0
paises               0
dtype: int64

In [28]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab" # aquí tendréis que poner vuestra contraseña de MySQL
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [29]:
mydb = crear_bbdd("tiburones")

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


2- Cread las tablas de la BBDD: tabla ataques, tabla clima

In [30]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [31]:
tabla_clima = """ CREATE TABLE IF NOT EXISTS clima (
  idclima INT NOT NULL AUTO_INCREMENT,
  timepoint INT NOT NULL,
  cloudcover INT NOT NULL,
  highcloud INT NOT NULL,
  midcloud  INT NOT NULL,
  lowcloud  INT NOT NULL,
  rh_profile VARCHAR(1000) NOT NULL,
  wind_profile VARCHAR(1000) NOT NULL,
  temp2m INT NOT NULL,
  lifted_index INT NOT NULL,
  rh2m INT NOT NULL,
  msl_pressure INT NOT NULL, 
  prec_type VARCHAR(255) NOT NULL,
  prec_amount INT NOT NULL,
  snow_depth INT NOT NULL,
  wind10m_direction INT NOT NULL,
  wind10m_speed INT NOT NULL,
  latitud FLOAT NOT NULL,
  longitud FLOAT NOT NULL,
  paises VARCHAR(255) NOT NULL,
  PRIMARY KEY (idclima))""" 

In [32]:
tabla_ataques = """ CREATE TABLE IF NOT EXISTS ataques (
  case_number VARCHAR(255) NOT NULL,
  year INT NOT NULL,
  country VARCHAR(255),
  activity VARCHAR(255),
  species_ VARCHAR(255),
  fatal_ VARCHAR(255),
  sex2 VARCHAR(255) NOT NULL,  
  nuevas_edades FLOAT NOT NULL,
  PRIMARY KEY (case_number))"""

In [33]:
crear_insertar_tabla("tiburones", "AlumnaAdalab", tabla_clima)

In [34]:
crear_insertar_tabla("tiburones", "AlumnaAdalab", tabla_ataques)

3- BONUS Insertar los datos en las tablas.

In [35]:
for indice, fila in df_clima.iterrows(): # itreamos por el dataframe
        
        query_clima = f"""
        
                INSERT INTO clima (timepoint, cloudcover, highcloud, midcloud, lowcloud, rh_profile, wind_profile, temp2m, lifted_index, rh2m, 
                msl_pressure, prec_type, prec_amount, snow_depth, wind10m_direction, wind10m_speed, latitud, longitud, paises) 
                
                VALUES ( "{fila["timepoint"]}","{fila['cloudcover']}", "{fila['highcloud']}", "{fila['midcloud']}", "{fila['lowcloud']}",
                "{fila['rh_profile']}","{fila['wind_profile']}","{fila['temp2m']}","{fila['lifted_index']}","{fila['rh2m']}","{fila['msl_pressure']}"
                ,"{fila['prec_type']}","{fila['prec_amount']}","{fila['snow_depth']}","{fila['wind10m.direction']}"
                ,"{fila['wind10m.speed']}","{fila['latitud']}","{fila['longitud']}","{fila['paises']}");
                """

    # una vez definida la query llamamos a la función que nos inserta los datos. 
        
        crear_insertar_tabla("tiburones", "AlumnaAdalab", query_clima)

In [36]:
for indice, fila in df_ataques.iterrows(): # itreamos por el dataframe
        
        query_ataques = f"""
        
                INSERT INTO ataques (case_number, year, country, activity, species_, fatal_, sex2, nuevas_edades)
                
                VALUES ( "{fila["case_number"]}","{fila['year']}","{fila['country']}","{fila['activity']}","{fila['species_']}","{fila['fatal_']}","{fila['sex2']}","{fila['nuevas_edades']}");
                """

    # una vez definida la query llamamos a la función que nos inserta los datos. 
        
        crear_insertar_tabla("tiburones", "AlumnaAdalab", query_ataques)